In [97]:
import pandas as pd
import plotly as pl

url = "data/combined_job_offers.csv"
url2 = "data/combined_job_offers3.csv"

df1 = pd.read_csv(url)
df2 = pd.read_csv(url2)

df3 = pd.concat([df1,df2],axis=0)

pd.set_option('display.max_columns', None)

In [99]:
#!pip install curl_cffi
import pandas as pd
from bs4 import BeautifulSoup
from curl_cffi import requests as cureq

headers = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
response = cureq.get("https://www.stepstone.de/jobs/data-analyst-or-data-analyst-e-commerce-or-data-analyst-specialist-or-business-data-analyst/in-berlin?radius=30&page=1&q=(Data+Analyst)+OR+(Data+Analyst+E-Commerce)+OR+(Data+Analyst+Specialist)+OR+(Business+Data+Analyst)", impersonate='chrome')
soup = BeautifulSoup(response.content, "html.parser")

In [101]:
# scrape job titles
itest = soup.find_all(['article'], attrs={"class": 'res-1p8f8en'})
job_title_list = []
for item in itest:
    job_title_list.append(item.find(['div'], attrs={'class': 'res-nehv70'}).get_text())

job_title_list

['Data Analyst Operations (d/w/m)',
 'Data Analyst (m/w/d)',
 'Business / Data Analyst (m/w/d)',
 'Werkstudent Data Engineer / Analyst (m/w/d)',
 'Business / Data Analyst*in (w/m/div)',
 'Data Analyst (m/w/d)',
 'Data Analyst (m/w/d)',
 'E-Commerce Cloud Solution Architect (f/m/x)',
 'Praktikant E-Commerce - Financial Accounting und Transaction Accounting Beratung - FAAS (w/m/d)',
 'Marketplace Manager / eCommerce Specialist (m/w/d)',
 'Mitarbeiter E-Commerce (m/w/d)',
 'Business Analyst (m/w/d) Data Analytics Vertriebsplanung | Banking',
 'Data Driven Business Consultant - Automotive (m/w/d)',
 'CRM Spezialist (m/w/d) Datenanalyse & Datenqualität',
 'Senior Data Analyst/ Senior Data Scientist',
 'Marktforscher (m/w/d) agri experts',
 'Research Engineer in AI / Data Scientist (m/f/d)',
 'Business Intelligence Expert:in',
 'Logistik-Analyst (m/w/d) für E-Commerce',
 'Physiker/ Mathematiker/ Naturwissenschaftler als Berater im Bereich Datenintegration (m/w/d)',
 'Data Scientist (mwd)',
 